In [3]:
import pickle
import pandas as pd
from pprint import pprint

# Path to your pickle file
file_path = 'zuco/all_trials.pkl'

try:
    # Open and load the pickle file
    with open(file_path, 'rb') as file:
        data = pickle.load(file)

    # Check the type of the loaded object
    print("Type of loaded object:", type(data))

    # Inspect the contents based on type
    if isinstance(data, dict):
        print("Keys in dictionary:", list(data.keys()))
        pprint(data, indent=2)
    elif isinstance(data, list):
        print("Length of list:", len(data))
        print("First few elements:", data[:5])
    elif isinstance(data, pd.DataFrame):
        print("DataFrame Info:")
        print(data.info())
        print("\nFirst few rows:")
        print(data.head())
    else:
        print("Contents:", data)

except FileNotFoundError:
    print(f"Error: The file {file_path} was not found.")
except pickle.UnpicklingError:
    print("Error: The file could not be unpickled. It may be corrupted or not a valid pickle file.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Type of loaded object: <class 'list'>
Length of list: 3747
First few elements: [(array([[ -6.205139  ,  -9.052857  ,   5.915986  , ..., -13.262074  ,
          0.4784965 ,   1.9660163 ],
       [ -4.4826202 ,  -6.509815  ,  -5.1510196 , ...,  -4.592909  ,
         -6.347468  , -11.518557  ],
       [ -0.09454989,  -1.1977538 ,  -3.3732831 , ...,  -2.9372294 ,
         -6.262914  ,  -6.4688706 ],
       ...,
       [-10.797141  ,  -8.944464  ,  -1.2070014 , ...,   1.5195919 ,
         -2.3751652 ,  -6.4546385 ],
       [ -4.6878133 ,  -1.4942873 ,  -3.720686  , ...,  -1.8454655 ,
         -5.115923  ,  -9.20201   ],
       [  3.2405083 ,   1.652156  ,  -3.0448663 , ...,   0.16234547,
         -2.9912531 ,   5.436593  ]], dtype=float32), 0), (array([[-6.2131734 , -8.3507395 ,  1.9430918 , ..., -4.6066265 ,
         4.612906  , 10.915043  ],
       [-3.1914546 , -1.0172254 ,  3.7683132 , ...,  1.5424702 ,
         5.601411  ,  6.2940106 ],
       [ 1.2527068 ,  1.475124  ,  4.3950095 , ..